In [1]:
!pip install pyDOE

In [2]:
#you need to add a dataset with GaussJacobiQuadRule_V3 on the right 
import sys
# sys.path.insert(1, '/kaggle/input/quadrule')
from GaussJacobiQuadRule_V3 import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights

#import GaussJacobiQuadRule_V3


import tensorflow as tf
#tf.disable_v2_behavior()
import pyDOE

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pyDOE import lhs
#from GaussJacobiQuadRule_V3.py import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights
import time

2023-09-12 16:46:03.056527: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-12 16:46:03.083888: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 16:46:03.291957: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-12 16:46:03.293055: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 16:46:04.108400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
class parameters:
    def __init__(self,LR,Opt_Niter,N_Element,N_testfcn,N_Quad,lossb_weight,a,b):
        self.LR = LR
        self.Opt_Niter = Opt_Niter + 1
        self.Opt_tresh = 2e-32
        self.var_form=2
        self.N_Element = N_Element
        self.N_testfcn = N_testfcn #number of test function
        self.N_Quad = N_Quad    #quadrature points 
        self.lossb_weight = lossb_weight #hyprparameter we may turn it up

        #params for the exact sol and boundary term
        self.omega = 8*np.pi
        self.amp = 1
        self.r1 = 80

        #ingredients for loss
        [self.x_quad, self.w_quad] = GaussLobattoJacobiWeights(N_Quad, 0, 0)
        print(self.x_quad.dtype)

        self.test_quad_element = self.Test_fcn(self.N_testfcn, self.x_quad)
        self.d1test_quad_element, self.d2test_quad_element = self.dTest_fcn(self.N_testfcn, self.x_quad)


        [self.x_l, self.x_r] = [a, b]     #modify in the future if you want general (a,b) interval
        self.delta_x = (self.x_r - self.x_l)/N_Element
        self.grid = np.asarray([ self.x_l + i*self.delta_x for i in range(self.N_Element+1)])

        #build right side (it doesnt depend on the network)

        self.F_ext_total = []
        for e in range(self.N_Element):
            x_quad_element = self.grid[e] + (self.grid[e+1]-self.grid[e])/2*(self.x_quad+1)  #traslation of the element 

            jacobian = (self.grid[e+1]-self.grid[e])/2
            testfcn_element = np.asarray([ self.Test_f(n,self.x_quad)  for n in range(1, self.N_testfcn+1)])

            #this is fh calculated on the real nodal values summed over each real nodal values(which comes form CGL nodes )

            f_quad_element = self.f_ext(x_quad_element)
            F_ext_element  = jacobian*np.asarray([sum(self.w_quad*f_quad_element*testfcn_element[i]) for i in range(self.N_testfcn)])    
            F_ext_element = F_ext_element[:,None] 
            self.F_ext_total.append(F_ext_element)

        self.F_ext_total = np.asarray(self.F_ext_total)


        self.X_bound = np.asarray([a,b],dtype=np.float64)[:,None]
        self.u_bound   = self.u_exact(self.X_bound)

    def u_exact(self,x):
        utemp = 0.1*np.sin(self.omega*x) + np.tanh(self.r1*x)
        return self.amp*utemp

    def f_ext(self,x):
        gtemp =  -0.1*(self.omega**2)*np.sin(self.omega*x) - (2*self.r1**2)*(np.tanh(self.r1*x))/((np.cosh(self.r1*x))**2)
        return -self.amp*gtemp

    def Test_f(self,n,x):
        test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
        return test
    
    def Test_fcn(self, N_test,x):
        test_total = []
        for n in range(1,N_test+1):  
            test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
            test_total.append(test)
        return np.asarray(test_total)

    def dTest_fcn(self, N_test,x):  #valuete the first and second derivatives of test functions on a point x 
        d1test_total = []
        d2test_total = []
        for n in range(1,N_test+1):  
            if n==1:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
            elif n==2:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
            else:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x) - ((n)*(n+1)/(2*2))*Jacobi(n-3,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
        return np.asarray(d1test_total), np.asarray(d2test_total)

In [4]:
params=parameters(0.01,1000,20,30,100,1,-1,1)

/home/mariano/Documenti/test/base/lib/python3.9/site-packages/scipy/special/_orthogonal.py:133: RuntimeWarning: invalid value encountered in multiply
  np.poly1d.__init__(self, poly.coeffs * float(kn))


float64


In [8]:
class Model(tf.keras.Model):
  def __init__(self,*kwargs):
    super().__init__()
    #structure of the nework maybe implment a general way
    self.dense1 = tf.keras.layers.Dense(units=16,
                                        activation='gelu',
                                        kernel_initializer=tf.random.normal,
                                        bias_initializer=tf.random.normal)
    self.dense2 = tf.keras.layers.Dense(units=16,
                                    activation='gelu',
                                    kernel_initializer=tf.random.normal,
                                    bias_initializer=tf.random.normal)
    self.y = tf.keras.layers.Dense(1)
    self.history=[]
    self.params=params
    
  def call(self, x, training=True):
    x = x[:, tf.newaxis]
    x = self.dense1(x)
    x = self.dense2(x)
    x = self.y(x)
    return tf.squeeze(x, axis=1)

  def net_du(self, x):
    t = tf.constant(x)
    with tf.GradientTape() as g:
      g.watch(x)
      with tf.GradientTape() as gg:
        gg.watch(x)
        y = self.call(x)
      dy_dx = gg.gradient(y, x)  # dy_dx = 2 * x
    d2y_dx2 = g.gradient(dy_dx, x)  # d2y_dx2 = 2
    return dy_dx,d2y_dx2

  def train(self):
    variables = self.variables
    optimizer = tf.optimizers.Adam(learning_rate=0.001)
    print('-->training_starting')
    for step in range(self.params.Opt_Niter):
      with tf.GradientTape() as tape:
        loss =self.calculate_loss()
        
      gradient = tape.gradient(loss, variables)
      optimizer.apply_gradients(zip(gradient, variables))

      if step % 100 == 0:
        print(f'Iter: {step}',f'Mean squared error: {loss.numpy():0.6f}')
        self.history.append(loss)
        
  def calculate_loss(self):
    
    varloss_total = 0
    
    for e in range(self.params.N_Element):
        F_ext_element  = self.params.F_ext_total[e]
        Ntest_element  = np.shape(F_ext_element)[0] #for each you have N_testfcn
        x_quad_element = tf.constant(self.params.grid[e] + (self.params.grid[e+1]-self.params.grid[e])/2*(self.params.x_quad+1))
        x_b_element    = tf.constant(np.array([[self.params.grid[e]], [self.params.grid[e+1]]]))
        #to change change integral to the ref segment in (-1,1)
        jacobian       = (self.params.grid[e+1]-self.params.grid[e])/2

        u_NN_quad_element = model(x_quad_element)
        
        d1u_NN_quad_element, d2u_NN_quad_element = self.net_du(x_quad_element)


    
        """"
        if self.params.var_form == 1:
            U_NN_element = tf.reshape(tf.stack([-jacobian*tf.reduce_sum(self.wquad*d2u_NN_quad_element*test_quad_element[i]) \
                                               for i in range(Ntest_element)]),(-1,1))
        """
        if self.params.var_form == 2:
            U_NN_element = tf.reshape(tf.stack([ tf.reduce_sum(self.params.w_quad*d1u_NN_quad_element*self.params.d1test_quad_element[i]) \
                                          for i in range(Ntest_element)]),(-1,1)) 
            #i think we are going to use this most of the times 
        """
        if self.params.var_form == 3:
            U_NN_element = tf.reshape(tf.stack([-1/jacobian*tf.reduce_sum(self.wquad*u_NN_quad_element*d2test_quad_element[i]) \
                                               +1/jacobian*tf.reduce_sum(u_NN_bound_element*np.array([-d1test_bound_element[i][0], d1test_bound_element[i][-1]]))  \
                                               for i in range(Ntest_element)]),(-1,1))
        """
        
        Res_NN_element = U_NN_element - F_ext_element
        loss_element = tf.reduce_mean(tf.square(Res_NN_element))
        varloss_total = varloss_total + loss_element
    

    lossb = tf.reduce_mean(tf.square(model(self.params.X_bound) - self.params.u_bound))  #u_NN_pred is what your network has calc,while u_tf is the real value at the boundary
    #two losses 
    

    
    return self.params.lossb_weight *lossb + tf.cast(varloss_total, tf.float32) #casting to fix bugs

In [9]:
model=Model(params)

In [10]:
model.train()

-->training_starting
Iter: 0 Mean squared error: 561.310730


KeyboardInterrupt: 

In [131]:
t=np.array([1,2,3])
model(t)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-5.346185, -6.553838, -7.684123], dtype=float32)>